In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import dateparser
from lxml import etree
from xml.etree.ElementTree import XML
'''
docxFileName = "quick_test.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
'''

'\ndocxFileName = "quick_test.docx"\ndocxZip = zipfile.ZipFile(docxFileName)\ndocumentXML = docxZip.read(\'word/document.xml\')\net = etree.XML(documentXML)\nns = {\'w\': \'http://schemas.openxmlformats.org/wordprocessingml/2006/main\'}\n'

In [2]:
'''
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
lowerstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="18"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def station_type (para):
    #if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.islower() == 'True':
        #return 'na'
    if para.xpath(mainstation_xpath, namespaces=ns):
        return 'main'        
    elif para.xpath(lowerstation_xpath, namespaces=ns):
        return 'lower'
    else:
        return 'na'


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'


for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text]) 
    if is_upper(para) == 'station':
        if station_type(para) =='main':
            mainstation = para.xpath(mainstation_xpath, namespaces=ns)            
            stations.at[i,'station'] = mainstation[0].text
            stations.at[i,'type'] = 'main'
            #print("MAIN:", mainstation[0].text)
        elif station_type(para) =='lower':
            lowerstation = para.xpath(lowerstation_xpath, namespaces=ns)
            stations.at[i,'station'] = lowerstation[0].text
            stations.at[i,'type'] = 'secondary'
            #print("SUB:", lowerstation[0].text)
    else:
        stations.at[i,'station'] = '-'
        stations.at[i,'type'] = '-'
    stations.at[i,'description'] = description
    #print("DESCRIPTION:", description)
    #print()


stations.head(3)
pd.options.display.max_colwidth = 100
#stations.to_csv('output.csv')


##notes: 
#"ABERDARE JUNCTION  see ABERCYNON." ("JUNCTION" in document.xml isn't marked-up correctly)
#"Bridge over line near Schoolhill ticket platform (ibid 8 June 1895). Separate structure or station used as collection point for..." ("Bridge" in document.xml isn't marked-up correctly)
'''
    


'\nmainstation_xpath = \'./w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]\'\nlowerstation_xpath = \'./w:r[1][not(w:rPr/w:sz[@w:val="18"])]/w:t[1]\'\nfirst_token_para_xpath = \'./w:r[1]/w:t[1]\'\n\nstations = pd.DataFrame(columns=[\'station\',\'type\',\'description\'])\n\ndef station_type (para):\n    #if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.islower() == \'True\':\n        #return \'na\'\n    if para.xpath(mainstation_xpath, namespaces=ns):\n        return \'main\'        \n    elif para.xpath(lowerstation_xpath, namespaces=ns):\n        return \'lower\'\n    else:\n        return \'na\'\n\n\ndef is_upper(para):\n    if para.xpath(first_token_para_xpath, namespaces=ns):\n        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():\n            return \'station\'\n        else: \n            return \'new_line\'\n\n\nfor i, para in enumerate(et.xpath(\'//w:p\', namespaces=ns)):\n    text = para.xpath(\'./w:r/w:t\', namespaces=ns)\n    description = " "

In [3]:
stations = pd.read_pickle("quick_processed.pkl")

In [4]:
stations.Description[0]

['A TOWN [NB] op 3 September 1856** as A; TOWN added 1889 tt (Cl) ; clo 7 September 1964 (RM October). Initially ABBEYHOLME (Brad).',
 'A JUNCTION   [Cal] op 31 August 1870  (co ½ T 26 September) . Clo 20 May 1921 (see 192 1  April/ May**).',
 'A JUNCTION   [NB] op 8 August 1870  (D&C 14) ; clo 1 September 1921 (Thursday)  (RCG)  – stops still shown until then in  Brad  ( all days August 1921)  but would seem to have been pointless after April/May 1921** .  L eft in case Cal line reopened? Did trains actually call?',
 'Brad  and co tts seen all have Abbey Junction for the station concerned. OS maps show as effectively one station.']

In [5]:
def find_between(s, start, end):
  return s.partition(start)[2].partition(end)[0]

def railway_company(s):
  pattern = re.compile('\[[a-zA-Z]+\]')
  m = pattern.findall(s)
  if m:
    return m[0].partition('[')[2].partition(']')[0]



def date_open(s):
  pattern = re.compile(r'op\s.*?\bclo\b')
  m = pattern.findall(s)
  if m:
     return m[0].partition('op')[2].partition('clo')[0].strip()
  else:
    pattern = re.compile(r'op\s.*?[0-9]{4}')
    m = pattern.findall(s)
    if m:
      return m[0].partition('op')[2].strip()


def date_closed(s):
  pattern = re.compile(r'clo\s.*?[.]')
  m = pattern.findall(s)
  if m:
     return m[0].partition('clo')[2].strip()
  else:
      pattern = re.compile(r'still open')
      m = pattern.findall(s)
      if m:
        return m[0].strip()


def place_disambiguation(s):
  pattern = re.compile(r'[A-Z]+\s*near\s*[a-zA-Z]*\s')
  m = pattern.findall(s)
  if m:
     return m[0].partition('near')[2].strip()


def additional_notes(s):
  end = date_closed(s)
  if end:
    notes = s.partition(str(end))[2]
    if notes:
      return notes.strip()

stations['place_disambiguation'] = stations.Description.apply(lambda x: place_disambiguation(x[0]))
stations['railway_company'] = stations.Description.apply(lambda x: railway_company(x[0]))
stations['open'] = stations.Description.apply(lambda x: date_open(x[0].lower()))
stations['closed'] = stations.Description.apply(lambda x: date_closed(x[0].lower()))
stations['additional_notes'] = stations.Description.apply(lambda x: additional_notes(x[0]))

#stations.to_csv('output.csv', encoding='utf-8')
stations.head(5)

,Station,Description,place_disambiguation,railway_company,open,closed,additional_notes
0,ABBEY,[A TOWN [NB] op 3 September 1856** as A; TOWN ...,None,NB,3 september 1856** as a; town added 1889 tt (c...,7 september 1964 (rm october).,Initially ABBEYHOLME (Brad).
1,ABBEY & WEST DEREHAM,[[GE] op 1 August 1882 (Thetford & Watton Time...,None,GE,1 august 1882 (thetford & watton times 5 th ) ...,22 september 1930 (cl) .,None
2,ABBEY FOREGATE,[– see SHREWSBURY.],None,None,None,None,None
3,ABBEY HOUSES,[see BEAUCHIEF .],None,None,None,None,None
4,ABBEY MILLS,[pumping station [private line south of West H...,None,None,None,None,None


In [7]:
#####TEST SANDBOX####

s =  "ABER near Caerphilly [Rhy] op 1 April 1908 (Aber) as BEDDAU (BEDA, error, in Brad); became A JUNCTION 17 September 1926 (GW circ 3011), A 6 May 1968 (Aberdare); HALT "
pattern = re.compile(r'[A-Z]+\s*near\s*[a-zA-Z]*\s')
m = pattern.findall(s)
if m:
    print(m[0])

ABER near Caerphilly 
